#### Preprocessing data


##### Filtering by specific postal code for spatial map plotting

In [9]:
import geopandas as gpd

geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\ADDRPT.geojson"
postal_code_112 = "090112"
postal_code_114 = "090114"

# Load the GeoJSON file
gdf = gpd.read_file(geojson_path)

# Function to retrieve coordinates by postal code
def get_coordinates_by_postal_code(postal_code):
    # Filter GeoDataFrame for the given postal code
    filtered_gdf = gdf[gdf['POSTAL_CODE'] == postal_code]
    if not filtered_gdf.empty:
        # Extract coordinates
        point = filtered_gdf.iloc[0].geometry
        return point.x, point.y
    else:
        return None, None

longitude_112, latitude_112 = get_coordinates_by_postal_code(postal_code_112)
longitude_114, latitude_114 = get_coordinates_by_postal_code(postal_code_114)

if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    print(f'Coordinates for postal code {postal_code_112}: Longitude {longitude_112}, Latitude {latitude_112}')
    print(f'Coordinates for postal code {postal_code_114}: Longitude {longitude_114}, Latitude {latitude_114}')
else:
    print('Postal code not found.')

Coordinates for postal code 090112: Longitude 103.82593292805574, Latitude 1.2745285256209595
Coordinates for postal code 090114: Longitude 103.82588719010951, Latitude 1.2750718182249274


In [10]:
# Finding the central postal code for Blocks 112 and 114
if longitude_112 and latitude_112 and longitude_114 and latitude_114:
    # Calculate the average coordinates
    avg_longitude = (longitude_112 + longitude_114) / 2
    avg_latitude = (latitude_112 + latitude_114) / 2
    print(f'Average coordinates between postal codes {postal_code_112} and {postal_code_114}:')
    print(f'Longitude: {avg_longitude}, Latitude: {avg_latitude}')
else:
    print('Postal code not found.')

Average coordinates between postal codes 090112 and 090114:
Longitude: 103.82591005908262, Latitude: 1.2748001719229434


In [48]:
# Converting x and y to coordinates for latitude/longitude
import rasterio
import numpy as np
import pandas as pd
from pyproj import Transformer
from shapely.geometry import Point

def preprocessing(file_path):   
    # Open your GeoTIFF file
    with rasterio.open(file_path) as src:
        array = src.read()
        transform = src.transform
        src_crs = src.crs  # Source CRS
        dest_crs = 'EPSG:4326'  # WGS 84

        # Create a transformer object to convert from src_crs to dest_crs
        transformer = Transformer.from_crs(src_crs, dest_crs, always_xy=True)

        # Get arrays of column and row indices
        cols, rows = np.meshgrid(np.arange(array.shape[2]), np.arange(array.shape[1]))
        
        # Convert meshgrid arrays to coordinate arrays using rasterio's method, which are 2D
        xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
        
        # Flatten the coordinate arrays to pass to transform function
        lon, lat = transformer.transform(np.array(xs).flatten(), np.array(ys).flatten())

        # Create DataFrame and convert to GeoDataFrame
        df = pd.DataFrame({'Longitude': lon, 'Latitude': lat})
        for i, band in enumerate(src.read(masked=True)):
            df[src.descriptions[i]] = band.flatten()

        # Convert 'SR_QA_AEROSOL' to integer for bitwise operation
        df['SR_QA_AEROSOL'] = df['SR_QA_AEROSOL'].astype(int)

        # Filter out pixels with valid aerosol retrieval and high aerosol level
        # Assuming 'SR_QA_AEROSOL' is the name of the QA aerosol band in the data
        valid_aerosol = (df['SR_QA_AEROSOL'] & 2) == 2  # Bit 1 must be set for valid retrieval
        high_aerosol = (df['SR_QA_AEROSOL'] & 192) == 192  # Bits 6-7 must be set to 11 for high aerosol
        filter_mask = valid_aerosol & high_aerosol
        df_filtered = df[-filter_mask]
        
        # Scale and offset specific bands
        df_filtered['ST_B10_Celsius'] = df_filtered['ST_B10'] * 0.00341802 + 149 - 273.15
        df_filtered = df_filtered[df_filtered['ST_B10_Celsius'] >= 20]  # Drop rows below 20 degrees Celsius
        
        bands_to_scale = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
        for band in bands_to_scale:
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * 2.75e-05 - 0.2

        additional_scales = {
            'ST_ATRAN': 0.0001, 'ST_CDIST': 0.01, 'ST_DRAD': 0.001, 
            'ST_EMIS': 0.0001, 'ST_EMSD': 0.0001, 'ST_QA': 0.01, 
            'ST_TRAD': 0.001, 'ST_URAD': 0.001
        }

        for band, scale in additional_scales.items():
            df_filtered[f"{band}_Scaled"] = df_filtered[band] * scale

        gdf = gpd.GeoDataFrame(df_filtered, geometry=gpd.points_from_xy(df_filtered.Longitude, df_filtered.Latitude))
        gdf.set_crs('EPSG:4326', inplace=True)  # Ensure the CRS is set to WGS 84

        print("Total number of valid pixels: " + str(len(gdf)))
        print(df[['Latitude', 'Longitude']].head())

        # Define your point of interest and buffer distance in meters
        poi = Point(avg_longitude, avg_latitude)
        desired_radius = 90
        buffer = poi.buffer(desired_radius / 111320)  # Convert meters to degrees approximately

        # Filter points within the buffer
        filtered_gdf = gdf[gdf.geometry.within(buffer)]

        # Save or process your filtered data
        print(f"\nNumber of points within {desired_radius}m radius: {len(filtered_gdf)}")
        #print(filtered_gdf['ST_B10_Celsius'].head())

    return filtered_gdf

#### Defining boundaries and plotting region of interest using GeoJSON

In [25]:
import geopandas as gpd

geojson_path = "C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\SG_geojson\\SG.geojson"

geo_data = gpd.read_file(geojson_path)

# Display the matching features
print(geo_data.columns)

Index(['osm_id', 'osm_type', 'addr_street', 'building', 'name', 'access_roof',
       'building_material', 'addr_housenumber', 'roof_material', 'geometry'],
      dtype='object')


In [26]:
matching_features = geo_data[(geo_data['addr_street'].str.contains("Bukit Purmei", na=False)) & 
                             (geo_data['addr_housenumber'].str.contains("112", na=False))]

if not matching_features.empty:
    polygon = matching_features.iloc[0]['geometry']
    print(polygon)
else:
    print("No matching features found.")

POLYGON ((103.8254384 1.2743995, 103.825484 1.2743955, 103.8254826 1.2743647, 103.8257254 1.274362, 103.8257254 1.2744009, 103.8260325 1.2743915, 103.8260754 1.2744344, 103.8262618 1.274429, 103.8263745 1.2745524, 103.8263785 1.2747656, 103.8262082 1.2747669, 103.8262045 1.274549, 103.8257106 1.2745598, 103.8257079 1.2745108, 103.8254424 1.2745122, 103.8254384 1.2743995))


In [101]:
import geopandas as gpd
import hvplot.pandas
import pandas as pd
from shapely.geometry import Polygon
import panel as pn
from bokeh.palettes import Inferno256 #Viridis256 / Turbo256
import logging

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

satellite_image = "L8_UTC_20201223_031657"
file_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\2020_test\\2020\\{satellite_image}.tif"

# Assuming preprocessing is defined elsewhere and correctly imports the file
filtered_gdf = preprocessing(file_path)

# Ensure the CRS is set
if filtered_gdf.crs is None:
    filtered_gdf.set_crs('epsg:4326', inplace=True)  # Assuming the data is in WGS 84

# Convert to Web Mercator for better mapping support
filtered_gdf = filtered_gdf.to_crs(epsg=3857)

# Create a GeoDataFrame for the Polygon (defined earlier)
polygon_gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs='epsg:3857')  # Set the CRS to Web Mercator

# Define a function to select a subset of the color palette
def select_colors(palette, n):
    return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

# Create a custom color scale using a continuous palette
custom_palette = select_colors(Inferno256, 256)  # More colors for smoother transitions

# Create the heatmap
heatmap = filtered_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=custom_palette, size=5, tiles='OSM', frame_width=700, frame_height=500, colorbar=True, clim=(20, 40))

# Plot the polygon with visible settings
polygon_plot = polygon_gdf.hvplot(geo=True, color='red', line_width=3, alpha=0.7)

# Overlay the polygon onto the heatmap
overlay_map = polygon_plot * heatmap

# Set up Panel to display the plot
pane = pn.panel(overlay_map)

pane.show()

Total number of valid pixels: 2151197
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 90m radius: 26
Launching server at http://localhost:52055


##### Filtering 30m x 30m pixels based on region of interest

##### Using ESPG:3857 allows you to blow up the pixels in metres because the coordinate representation is in metres

In [47]:
import geopandas as gpd
import hvplot.pandas
import pandas as pd
from shapely.geometry import Polygon, box
import panel as pn
from bokeh.palettes import Inferno256
import numpy as np
import logging

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

satellite_image = "L8_UTC_20201223_031657"
file_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\2020_test\\2020\\{satellite_image}.tif"

# Assuming preprocessing is defined elsewhere and correctly imports the file
filtered_gdf = preprocessing(file_path)

def plot_spatial_map(filtered_gdf): 
    # Ensure the CRS is set
    if filtered_gdf.crs is None:
        filtered_gdf.set_crs('epsg:4326', inplace=True)  # Assuming the data is in WGS 84

    filtered_gdf = filtered_gdf.to_crs('epsg:3857')

    # print(filtered_gdf['geometry'])

    # Create pixels as 30m x 30m boxes around each point
    # Assuming each point is at the center of the pixel
    half_width = 15  # half the width of the pixel in meters since the ESPG:3857 coordinate system is in metres
    filtered_gdf['geometry'] = filtered_gdf['geometry'].apply(lambda x: box(x.x - half_width, x.y - half_width, x.x + half_width, x.y + half_width))

    #print(filtered_gdf['geometry'])

    # Create a GeoDataFrame for the Polygon (defined earlier)
    polygon_gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs='epsg:4326')  # Assuming the original CRS is EPSG 4326 (WGS84)

    # Convert the geometry to EPSG 3857
    polygon_gdf_3857 = polygon_gdf.to_crs(epsg=3857)

    # print(polygon_gdf_3857['geometry'])

    # Choosing pixels that intersect with the polygon
    pip = filtered_gdf.intersects(polygon_gdf_3857.loc[0, 'geometry'])

    # Creating a new GeoDataFrame that will have only the intersecting pixels
    within_polygon_gdf = filtered_gdf.loc[pip].copy()

    # Resetting index (optional)
    within_polygon_gdf.reset_index(inplace=True, drop=True)

    # Print or use the filtered GeoDataFrame as needed
    # print("\nNumber of points within the region of interest: " + str(len(within_polygon_gdf)))

    # # Print the centroids of the intersected pixels
    # for index, row in within_polygon_gdf.iterrows():
    #     centroid = row['geometry'].centroid
    #     print(f"Longitude: {centroid.x}, Latitude: {centroid.y}")

    # Define a function to select a subset of the color palette
    def select_colors(palette, n):
        return [palette[int(i)] for i in np.linspace(0, len(palette)-1, n)]

    # Create a custom color scale using a continuous palette
    custom_palette = select_colors(Inferno256, 256)  # More colors for smoother transitions

    # Create the heatmap using the centroid points of intersected pixels
    heatmap = within_polygon_gdf.hvplot.points('Longitude', 'Latitude', geo=True, c='ST_B10_Celsius', cmap=custom_palette, size=5, tiles='OSM', frame_width=700, frame_height=500, colorbar=True, clim=(20, 40))

    # Plot square polygons with the same color mapping as the points
    squares_plot = within_polygon_gdf.hvplot.polygons('geometry', c='ST_B10_Celsius', cmap=custom_palette, alpha=0.5, colorbar=True, clim=(20, 40))

    # Plot the polygon with visible settings
    polygon_plot = polygon_gdf.hvplot(geo=True, color='red', line_width=3, alpha=0.7)

    # Overlay the polygon onto the heatmap
    overlay_map = polygon_plot * heatmap * squares_plot

    # Set up Panel to display the plot
    pane = pn.panel(overlay_map)

    # pane.show()
    # pane.save(f'C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\MSE-ES-UHI\\2_landsat\\Heatmaps\\{postal_code_112}_{satellite_image}_LST_Filtered.html', embed=True)

    return overlay_map

Total number of valid pixels: 2151197
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 90m radius: 26


#### Plotting LST over time

##### Combining GDFs

In [49]:
import geopandas as gpd
import pandas as pd
import os
import zipfile
from datetime import datetime
import logging
import shutil

year = "2020"

# Suppress warnings
logging.getLogger('bokeh').setLevel(logging.ERROR)
pd.options.mode.chained_assignment = None  # default='warn'

# Specify the zip file and temporary directory for extraction
zip_file_path = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\Landsat8\\{year}.zip"
temp_dir = f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract"

# Create a temporary directory if it doesn't exist
os.makedirs(temp_dir, exist_ok=True)

# Extract the .tif files from the zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Initialize an empty list to hold all the GeoDataFrames
gdfs = []

# Walk through the temporary directory and process each .tif file
for filename in os.listdir(f"{temp_dir}\\{year}"):
    if filename.endswith(".tif"):
        print("Currently processing: " + filename)
        file_path = os.path.join(f"{temp_dir}\\{year}", filename)
        
        # Extract the time period from the filename
        # Assuming filename format is "L8_UTC_YYYYMMDD_hhmmss.tif"
        time_str = filename.split('_')[2]
        time_obj = datetime.strptime(time_str, "%Y%m%d")
        
        # Load and preprocess the GeoDataFrame
        gdf = preprocessing(file_path)
        gdf['time'] = time_obj  # Append the datetime object as a new column
        
        # Append the processed GeoDataFrame to the list
        gdfs.append(gdf)

# Combine all GeoDataFrames into one
combined_gdf = pd.concat(gdfs)

shutil.rmtree(f"C:\\LocalOneDrive\\Documents\\Desktop\\MTI\\UHI-Project\\MSE-ES-UHI\\Data\\temp_extract")

# Use the combined GeoDataFrame as needed
print(combined_gdf)

Currently processing: L8_UTC_20200106_031650.tif
Total number of valid pixels: 1912476
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 90m radius: 27
Currently processing: L8_UTC_20200122_031646.tif
Total number of valid pixels: 1731121
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 90m radius: 28
Currently processing: L8_UTC_20200207_031641.tif
Total number of valid pixels: 1216672
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.470099  103.590560
4  1.470100  103.590830

Number of points within 90m radius: 0
Currently processing: L8_UTC_20200223_031638.tif
Total number of valid pixels: 1345446
   Latitude   Longitude
0  1.470099  103.589751
1  1.470099  103.590021
2  1.470099  103.590290
3  1.

##### Spatial plot over time

In [50]:
import panel as pn

# Create an interactive plot with filtering based on the GeoDataFrame
def create_interactive_plot(combined_gdf):
        # Create a list of unique dates sorted
    unique_dates = combined_gdf['time'].dt.strftime('%Y-%m-%d').sort_values().unique()
    # print(f"Unique Dates: {unique_dates}")

    date_index_map = {i + 1: date for i, date in enumerate(unique_dates)}

    # Setup an integer slider to select time periods
    time_slider = pn.widgets.IntSlider(name='Select Time', start=1, end=len(unique_dates), value=1, step=1)

    @pn.depends(time_slider.param.value_throttled)
    def dynamic_map(value):
        selected_date = date_index_map[value]
        selected_datetime = pd.to_datetime(selected_date).date()
        
        # Filter data for the selected time
        filtered_data = combined_gdf[combined_gdf['time'].dt.date == selected_datetime]

        # Call plot_spatial_map for the selected time period
        return plot_spatial_map(filtered_data)

    layout = pn.Column(
        "<br>\nInteractive Land Surface Temperature Map",
        time_slider,
        dynamic_map
    )

    return layout

# Assuming 'combined_gdf' is a GeoDataFrame already loaded and ready to be used.
layout = create_interactive_plot(combined_gdf)
# layout.servable()
pn.serve(layout, show=False, start=True)

Launching server at http://localhost:59368
